# Capstone: Recommender System
## 2. Data Cleaning and Pre-processing

In [1]:
import pandas as pd
import numpy as np
import regex as re

In [131]:
#part 1 product info
df = pd.read_csv('../datasets/product_info_raw.csv')

In [132]:
df.head()

,Unnamed: 0,Category,URL,brand,name,rating,price,descriptions,ingredients,pic,productsize
0,0,face-wash-facial-cleanser,https://www.sephora.com/product/soy-face-clean...,FRESH,Soy Face Cleanser,4.4,$38.00,"What it is: A bestselling, gently powerful fac...",-Amino Acid-rich Soy Proteins: Help maintain s...,https://www.sephora.com/productimages/sku/s487...,SIZE: 5.1 oz/ 150 mL
1,1,face-wash-facial-cleanser,https://www.sephora.com/product/kale-spinach-g...,YOUTH TO THE PEOPLE,Superfood Antioxidant Cleanser,4.3,$36.00,What it is: A daily green juice cleanse for yo...,"Water, Sodium Cocoyl Glutamate, Cocamidopropyl...",https://www.sephora.com/productimages/sku/s186...,SIZE: 8 oz/ 237 mL
2,2,face-wash-facial-cleanser,https://www.sephora.com/product/green-clean-ma...,FARMACY,Green Clean Makeup Removing Cleansing Balm,4.6,$34.00,What it is: An award-winning makeup remover an...,-Sunflower and Ginger Root Oils: Melt even stu...,https://www.sephora.com/productimages/sku/s189...,SIZE: 3.2 oz/ 90 mL
3,3,face-wash-facial-cleanser,https://www.sephora.com/product/sugar-strawber...,FRESH,Sugar Strawberry Exfoliating Face Wash,4.5,$33.00,What it is: A gentle exfoliating face cleanser...,-Blend of White Sugar Crystals: Help smooth an...,https://www.sephora.com/productimages/sku/s222...,SIZE 4.2 oz/ 125 mL•ITEM 2223592
4,4,face-wash-facial-cleanser,https://www.sephora.com/product/purity-made-si...,PHILOSOPHY,Purity Made Simple Cleanser,4.5,$24.00,"What it is: A face wash for easy, one-step cle...",-Sodium Lauroamphoacetate: Acts as a mild surf...,https://www.sephora.com/productimages/sku/s407...,SIZE: 8 oz


In [133]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [134]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 10 columns):
Category        288 non-null object
URL             288 non-null object
brand           288 non-null object
name            288 non-null object
rating          285 non-null float64
price           288 non-null object
descriptions    288 non-null object
ingredients     288 non-null object
pic             288 non-null object
productsize     287 non-null object
dtypes: float64(1), object(9)
memory usage: 22.6+ KB


In [135]:
#drop rows contains NaN 
df = df[~df.rating.isnull()]
df = df[~df.productsize.isnull()]

In [136]:
#clean price column
p = [i.split('\n')[0] for i in df.price]
p = [i.split(' ')[0] for i in p]
df.price = [float(i.replace('$','')) for i in p]

In [ ]:
#clean product size column

size_oz1=[re.findall("(\d+\.*\d*) oz",i) for i in df.productsize]
size_oz2 = []
for i in range(len(size_oz1)):
    if size_oz1[i] != []:
        try:
            size_oz1[i] = float(size_oz1[i][0])
            size_oz2.append(size_oz1[i])
        except ValueError:
            n = re.findall("(\d+)",size_oz1[i])
            size_oz2.append(n)
    else:
        print(df.iloc[i],'\n *********************')
        size_oz2.append(np.nan)
   

In [138]:
#adding new column oz to df
df = df.assign(oz = size_oz2)

In [ ]:
#fill in value 
df.oz[12] = 3.88
df.oz[32] = 2.00
df.oz[59] = 1.00
df.oz[68] = 2.00
df.oz[81] = 1.00
df.oz[88] = 4.22
df.oz[92] = 5.00
df.oz[152] = 8.40
df.oz[174] = 0.90
df.oz[175] = 6.50
df.oz[183] = 2.00
df.oz[207] = 1.08
df.oz[215] = 1.70
df.oz[219] = 1.60
df.oz[226] = 1.00
df.oz[237] = 1.70
df.oz[244] = 4.20
df.oz[252] = 0.37
df.oz[259] = 1.70
df.oz[268] = 1.70
df.oz[271] = 1.69
df.oz[274] = 1.69



In [143]:
df = df[~df.oz.isnull()]

In [166]:
index = list(df.index)

In [167]:
#combine product brand and product name to create new column full_name
df.full_name = ''
for i in index:
    df.full_name[i] = df.brand[i] +' '+ df.name[i]
    

/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [171]:
#based on size and price of product, add a new column price per oz called price_oz
price=[]
for i in index:
    num = df.price[i]/df.oz[i]
    price.append(num)
df = df.assign(price_oz = price)

In [212]:
#ingredients 
l = []
for i in index:
    try:
        e = df.ingredients[i].split('\n\n')[1]
        l.append(e)
    except:
        l.append(df.ingredients[i])
df = df.assign(pure_ingredients = l)

In [ ]:
#manully fill in ingredients
df.pure_ingredients[12] = df.ingredients[12].split('\n\n')[0].replace('Slaai™ Makeup-Melting Butter Cleanser:\n','')
df.pure_ingredients[32] = df.ingredients[32].replace('Pekee Bar™:\n','')
df.pure_ingredients[46] = df.ingredients[46].replace('Radiant Vitality 2 In 1 Foam Clnsr Division: El (Estee Lauder)Ingredients: ','')
df.pure_ingredients[67] = 'Aqua (Water), Cetearyl Ethylhexanoate, Cetearyl Alcohol, Caprylic/ Capric Triglyceride, Sesamum Indicum (Sesame) Seed Oil, Cetearyl Glucoside, Butyrospermum Parkii (Shea Butter), Myristyl Myristate, Glycerin, Ribes Nigrum (Black currant) Seed Oil, Phenoxyethanol, Helianthus Annuus (Sunflower) Seed Oil, Oryzanol, Bisabolol, Sodium Hydroxymethylglycinate, Hippophae Rhamnoides Fruit Extract, Calendula Officinalis Flower Extract, Citric Acid, Tocopherol, Xanthan Gum, Limonene, Linalool, Anthemis Nobilis Flower Oil, Pelargonium Graveolens Flower Oil, Foeniculum Vulgare (Fennel) Fruit Extract, Citronellol, Citrus Nobilis (Mandarin Orange) Peel Oil, Anthemis Nobilis Flower Extract, Cinnamomum Camphora Linalloliferum (Ho Wood) Leaf Oil.'
df.pure_ingredients[68] = df.ingredients[68].replace('Pekee Bar™: ','')
df.pure_ingredients[95] = 'AQUA/WATER/EAU, COCO-GLUCOSIDE, DISODIUM LAURYL SULFOSUCCINATE, GLYCERIN, COCO-BETAINE, PROPANEDIOL, SODIUM LAUROYL GLUTAMATE, DISODIUM COCOAMPHODIACETATE, ACRYLATES COPOLYMER, ALOE BARBADENSIS LEAF EXTRACT, ASCORBYL TETRAISOPALMITATE, BENZOIC ACID, BIOFLAVONOIDS, BRASSICA OLERACEA ITALICA(BROCCOLI) EXTRACT, BUTYLENE GLYCOL, CALENDULA OFFICINALIS FLOWER EXTRACT, CITRIC ACID, GLYCERYL OLEATE, GLYCERYL STEARATE, GLYCOL DISTEARATE, HELIANTHUS ANNUUS (SUNFLOWER)SEED OIL, LACTIC ACID, LONICERA CAPRIFOLIUM (HONEYSUCKLE) FLOWER EXTRACT, LONICERA JAPONICA (HONEYSUCKLE) FLOWER EXTRACT, PANTHENOL, PARFUM/FRAGRANCE, PEG-120 METHYL GLUCOSE DIOLEATE, PHENOXYETHANOL, POTASSIUM SORBATE, ROSA CANINA FRUIT OIL, ROSMARINUS OFFICINALIS (ROSEMARY) EXTRACT, SIDERITIS SCARDICA FLOWER/LEAF/STEM EXTRACT, SODIUM BENZOATE, SODIUM CHLORIDE, SODIUM HYDROXIDE, TETRASODIUM GLUTAMATE DIACETATE, TOCOPHEROL.'
df.pure_ingredients[117] = 'Aqua (Water, Eau), Glycerin, Hamamelis Virginiana (Witch Hazel) Leaf Extract, Rosa Damascena (Rose) Flower Water, Aloe Barbadensis Leaf Juice, Rosa Hybrid Flower Extract, Rosa Domascena Flower Oil, Pentylene Glycol, Ethylhexylglycerin, Propanediol, Polysorbate 20, Potassium Sorbate, Parfum (Fragrance), Sodium Benzoate, Sodium Chloride, Citric Acid, Phenoxyethanol, Benzoic Acid, CI 16035 (Red 40), CI 15985 (Yellow 6), CI 19140 (Yellow 5)'
df.pure_ingredients[148] = 'Water, Glycerin, Sodium Citrate, Ci 42090/ Blue 1, Ci 14700/ Red 4, Peg-60 Hydrogenated Castor Oil, Chlorphenesin, Sambucus Nigra Flower Water, Linalool, Benzyl Alcohol, Propylparaben, Alpha-Isomethyl Ionone, Geraniol, Rosa Centifolia Flower Water, Rosa Centifolia Water/ Rosa Centifolia Flower Water, Methylparaben, Tetrasodium Edta, Citronellol, Hexylene Glycol, Fragrance.'
df.pure_ingredients[162] = 'Aqua/Water, Pentylene Glycol, Phenoxyethanol, Propylene Glycol, Sodium PCA, Methylparaben, Allantoin, Cucumis Sativus Fruit Extract/Cucumber Fruit Extract, Aloe Barbadensis/Aloe Barbadensis Leaf Juice, Camphor, Chamomilla Recutita Extract/Matricaria Flower Extract, Urtica Dioica/Nettle Extract, Equisetum Arvense Extract, Betula Alba Extract, CI 19140/Yellow 5, CI 14700/Red 4, CI 42090/Blue 1.'
df.pure_ingredients[169] = np.nan
df.pure_ingredients[173] = 'Water, Aloe Barbadensis Leaf Juice, Dipropylene Glycol, PEG-8, Propylene Glycol, Linum Usitatissimum (Linseed) Seed Extract, Phenoxyethanol, Panthenol, PEG-32, PPG-26-Buteth-26, Alpha-Glucan Oligosaccharide, Sodium Citrate, Triethanolamine, PEG-40 Hydrogenated Castor Oil, Methylparaben, Acrylates/C10-30 Alkyl Acrylate Crosspolymer, Fragrance, Glycerin, Potassium Sorbate, Polyacrylic Acid, Butylene Glycol, Tocopheryl Acetate, Althea Officinalis Root Extract, Prunus Amygdalus Dulcis (Sweet Almond) Fruit Extract, Retinyl Palmitate, Arachis Hypogaea (Peanut) Oil, Hydrolyzed Linseed Extract, Propylparaben, Butylparaben, Ethylparaben, Tilia Cordata Wood Extract, Sodium Benzoate, Isobutylparaben, Xanthan Gum, Chlorphenesin, BHT, Alpha-Isomethyl Ionone, Benzyl Alcohol, Amyl Cinnamal, Anise Alcohol, Benzyl Benzoate, Eugenol, Yellow 6'
df.pure_ingredients[175] = 'Water\Aqua\Eau , Alcohol Denat. , Glycerin , Dipropylene Glycol , Propanediol , Trehalose , Cucumis Sativus (Cucumber) Fruit Extract , Paeonia Suffruticosa (Peony) Root Extract , Sapindus Mukorossi Fruit Extract , Panax Ginseng (Ginseng) Root Extract , Citrullus Vulgaris (Watermelon) Fruit Extract , Vaccinium Macrocarpon (Cranberry) Fruit Extract , Vaccinium Angustifolium (Blueberry) Fruit Extract , Lycium Chinense (Wolfberry) Fruit Extract , Punica Granatum (Pomegranate) Extract , Pyrus Malus (Apple) Fruit Extract , Lens Esculenta (Lentil) Fruit Extract , Yeast Extract\Faex\Extrait De Levure , Algae Extract , Caesalpinia Spinosa Gum , Sodium Lactate , Sodium Hyaluronate , Tocopheryl Acetate , Caffeine , Butylene Glycol , Ergothioneine , Acetyl Glucosamine , Glycine Soja (Soybean) Seed Extract , Tetrahexyldecyl Ascorbate , Peg-8 , Ppg-6-Decyltetradeceth-30 , Carbomer , Tromethamine , Citric Acid , Sodium Pca , Fragrance (Parfum) , Trisodium Hedta , Phenoxyethanol , Yellow 6 (Ci 15985) , Red 4 (Ci 14700)'
df.pure_ingredients[178] = df.ingredients[178].replace('Perfectly Clean Mlt-Act Tng Ltn/Rf Division: El (Estee Lauder)Ingredients: ','')
df.pure_ingredients[183] = 'Water (Aqua Purificata) Purified, Alcohol Denat., Hamamelis Virginiana (Witch Hazel) Water, Aloe Barbadensis Leaf Juice, Butylene Glycol, Glycerin, Polysorbate 20, Beta-Glucan, Avena Sativa (Oat) Kernel Extract, Potassium Azeloyl Diglycinate, Sodium Hyaluronate, Allantoin, Panthenol, Mentha Piperita (Peppermint) Leaf Oil, Eucalyptus Globulus (Eucalyptus) Leaf Oil, Rosemarinus Officinalis (Rosemary) Leaf Oil, Citrus Grandes (Grapefruit) Peel Oil, Caprylyl Glycol, Isopentyldiol, Phenoxyethanol, Tetrasodium EDTA, Yellow 5 (CI 19140), Blue 1 (CI 42090).'
df.pure_ingredients[194] = df.ingredients[194].split('\n\n')[0]
df.pure_ingredients[199] = df.ingredients[199].split('\n\n')[0]
df.pure_ingredients[200] = 'Virgin Marula Oil'
df.pure_ingredients[203] = df.ingredients[203].split('\n\n')[0]
df.pure_ingredients[205] = df.ingredients[205].split('\n\n')[0]
df.pure_ingredients[217] = df.ingredients[217].split('\n\n')[0]
df.pure_ingredients[254] = 'AQUA (WATER), CYCLOPENTASILOXANE, GLYCERIN, ALCOHOL, POLYSILICONE-11, POLYMETHYLSILSESQUIOXANE, PENTAERYTHRITYL TETRAISOSTEARATE, CI 77163 (BISMUTH OXYCHLORIDE), CETYL PEG/PPG-10/1 DIMETHICONE, BUTYLENE GLYCOL, SYNTHETIC FLUORPHLOGOPITE, CI 77891 (TITANIUM DIOXIDE), PHENOXYETHANOL, MAGNESIUM SULFATE, SORBITAN SESQUIOLEATE, DIPOTASSIUM GLYCYRRHIZATE, DISTEARDIMONIUM HECTORITE, XYLITOL, PENTYLENE GLYCOL, PARFUM (FRAGRANCE), SECALE CEREALE (RYE) SEED EXTRACT, ASCORBYL GLUCOSIDE, PROPYLENE CARBONATE, HEXADECENE, MALVA SYLVESTRIS (MALLOW) EXTRACT, ALUMINUM HYDROXIDE, CI 77492 (IRON OXIDES), ADENOSINE, SAPONINS, CI 77491 (IRON OXIDES), STEARIC ACID, HYDROGEN DIMETHICONE, BUTYLPHENYL METHYLPROPIONAL, BHT, LIMONENE, PALMARIA PALMATA EXTRACT, AFRAMOMUM ANGUSTIFOLIUM SEED EXTRACT, SODIUM BENZOATE, HYDROLYZED RICE PROTEIN, ALPHA-ISOMETHYL IONONE, SODIUM MYRISTOYL GLUTAMATE, GERANIOL, JASMINUM OFFICINALE (JASMINE) FLOWER EXTRACT, POLYGONUM BISTORTA ROOT EXTRACT, PENTAERYTHRITYL TETRA-DI-T-BUTYL HYDROXYHYDROCINNAMATE, TOCOPHEROL'
df.pure_ingredients[256] = df.ingredients[256].replace('Revitalizing Supreme+ Cell Pwr Crm Division: El (Estee Lauder)Ingredients: ','')
df.pure_ingredients[264] = 'Avobenzone 2.2%, Octinoxate 7.5%, Water, Cyclopentasiloxane, Octyldodecyl Neopentanoate, Phenethyl Benzoate, Haslea Ostrearia Extract , Palmaria Palmata Extract, Sodium Hyaluronate, Tocopheryl Acetate, Retinyl Palmitate, Lecithin, Glycerin, Pyrus Malus (Apple) Fruit Extract, Lavandula Angustifolia (Lavender) Oil, Rosmarinus Officinalis (Rosemary) Leaf Oil, Melissa Officinalis (Balm Mint) Extract, Calendula Officinalis Extract*, Leontopodium Alpinum (Edelweiss) Extract*, Peucedanum Ostruthium Leaf Extract*, Buddleja Davidii Leaf Extract*, Artemisia Umbelliformis Extract *, Anthemis Nobilis Flower Oil, Pelargonium Graveolens Flower Oil, Dimethicone, Glycereth-26, Hydroxyethyl Acrylate/Sodium Acryloyldimethyl Taurate Copolymer, Phenoxyethanol, Hydroxypropyl Starch Phosphate, Cetearyl Glucoside, Carbomer, Butylene Glycol, Sodium Hydroxide, Disodium EDTA, Iodopropynyl Butylcarbamate.'
df.pure_ingredients[273] = df.ingredients[273].split('\n\n')[0]



In [306]:
#rearrange column names
df = df[['full_name','brand', 'name', 'Category', 'URL', 'rating', 'price', 'productsize', 'oz', 'price_oz', 'descriptions', 'ingredients', 'pure_ingredients','pic']]

In [307]:
#drop products without ingredients after cleaning
df = df[~df.pure_ingredients.isnull()]

In [308]:
df.to_csv('product_info_clean.csv')

In [290]:
#total 281 products left
len(df)

281

In [ ]:
#some products have the same name, we are going to rename them
df.full_name.value_counts()

In [315]:
df[df.full_name == 'CLINIQUE Clarifying Lotion 3']

,full_name,brand,name,Category,URL,rating,price,productsize,oz,price_oz,descriptions,ingredients,pure_ingredients,pic
114,CLINIQUE Clarifying Lotion 3,CLINIQUE,Clarifying Lotion 3,facial-toner-skin-toner,https://www.sephora.com/product/clarifying-lot...,4.4,16.0,SIZE: 6.7 oz/ 200 mL,6.7,2.388060,"What it is:\nA gentle, oil-free exfoliating lo...","Water , Alcohol Denat. , Salicylic Acid , Hama...","Water , Alcohol Denat. , Salicylic Acid , Hama...",https://www.sephora.com/productimages/sku/s129...
139,CLINIQUE Clarifying Lotion 3,CLINIQUE,Clarifying Lotion 3,facial-toner-skin-toner,https://www.sephora.com/product/clarifying-lot...,4.2,29.5,SIZE: 16.5 oz/ 487 mL,16.5,1.787879,"What it is:\nA gentle, oil-free exfoliating lo...","Water, Alcohol Denat., Salicylic Acid, Hamamel...","Water, Alcohol Denat., Salicylic Acid, Hamamel...",https://www.sephora.com/productimages/sku/s216...


In [317]:
df.loc[139,'full_name'] = 'CLINIQUE Clarifying Lotion 3 v2'

In [318]:
df[df.full_name == 'CLINIQUE Clarifying Lotion 2']

,full_name,brand,name,Category,URL,rating,price,productsize,oz,price_oz,descriptions,ingredients,pure_ingredients,pic
107,CLINIQUE Clarifying Lotion 2,CLINIQUE,Clarifying Lotion 2,facial-toner-skin-toner,https://www.sephora.com/product/clarifying-lot...,4.4,16.0,SIZE: 6.7 oz/ 200 mL,6.7,2.388060,"What it is:\nA gentle, oil-free exfoliating lo...","Water , Alcohol Denat. , Glycerin , Hamamelis ...","Water , Alcohol Denat. , Glycerin , Hamamelis ...",https://www.sephora.com/productimages/sku/s129...
129,CLINIQUE Clarifying Lotion 2,CLINIQUE,Clarifying Lotion 2,facial-toner-skin-toner,https://www.sephora.com/product/clarifying-lot...,4.4,29.5,SIZE: 16.5 oz/ 487 mL,16.5,1.787879,"What it is:\nA gentle, oil-free exfoliating lo...","Water, Alcohol Denat., Glycerin, Hamamelis Vir...","Water, Alcohol Denat., Glycerin, Hamamelis Vir...",https://www.sephora.com/productimages/sku/s203...


In [319]:
df.loc[129,'full_name'] = 'CLINIQUE Clarifying Lotion 2 v2'

In [320]:
df[df.full_name == 'BELIF Witch Hazel Herbal Extract Toner']

,full_name,brand,name,Category,URL,rating,price,productsize,oz,price_oz,descriptions,ingredients,pure_ingredients,pic
56,BELIF Witch Hazel Herbal Extract Toner,BELIF,Witch Hazel Herbal Extract Toner,face-wash-facial-cleanser,https://www.sephora.com/product/witch-hazel-he...,4.6,28.0,SIZE 6.75 oz/ 200 mL•ITEM 1686344,6.75,4.148148,What it is:\nA toner formulated with witch haz...,"Water, Monarda Didyma Leaf Extract 8%, Glyceri...","Water, Monarda Didyma Leaf Extract 8%, Glyceri...",https://www.sephora.com/productimages/sku/s348...
116,BELIF Witch Hazel Herbal Extract Toner,BELIF,Witch Hazel Herbal Extract Toner,facial-toner-skin-toner,https://www.sephora.com/product/witch-hazel-he...,4.6,28.0,SIZE 6.75 oz/ 200 mL•ITEM 1686344,6.75,4.148148,What it is:\nA toner formulated with witch haz...,"Water, Monarda Didyma Leaf Extract 8%, Glyceri...","Water, Monarda Didyma Leaf Extract 8%, Glyceri...",https://www.sephora.com/productimages/sku/s348...


In [321]:
df.loc[116,'full_name'] = 'BELIF Witch Hazel Herbal Extract Toner v2'

In [323]:
df.to_csv('product_info_clean.csv')

In [2]:
#part 2 user reviews 
review = pd.read_csv("../datasets/review.csv")

In [3]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158208 entries, 0 to 158207
Data columns (total 11 columns):
Unnamed: 0      158208 non-null int64
Unnamed: 0.1    158208 non-null int64
Category        158208 non-null object
brand           158208 non-null object
name            158208 non-null object
user            156907 non-null object
stars           158208 non-null int64
short_review    122493 non-null object
long_review     155827 non-null object
helpfulness     158208 non-null int64
time            157666 non-null object
dtypes: int64(4), object(7)
memory usage: 13.3+ MB


In [5]:
review.drop(columns = ['Unnamed: 0','Unnamed: 0.1'], inplace = True)
full_name = review.brand+' '+review.name
df_fn = full_name.to_frame(name = 'full_name')
review = pd.concat([df_fn,review],axis= 1)
review.head()

,full_name,Category,brand,name,user,stars,short_review,long_review,helpfulness,time
0,FRESH Soy Face Cleanser,face-wash-facial-cleanser,FRESH,Soy Face Cleanser,camha,1,No thank you,This cleanser has too many preservatives in it...,276,26 Oct 2013
1,FRESH Soy Face Cleanser,face-wash-facial-cleanser,FRESH,Soy Face Cleanser,ChristineJi,1,irritates my skin,I originally received a sample size of this cl...,109,21 Dec 2016
2,FRESH Soy Face Cleanser,face-wash-facial-cleanser,FRESH,Soy Face Cleanser,tracyacnp,4,"Good product, but a little pricey","Quality wise, this is an awesome product. My f...",49,9 Oct 2008
3,FRESH Soy Face Cleanser,face-wash-facial-cleanser,FRESH,Soy Face Cleanser,tenngal,5,Excellent cleanser...,Fresh's Soy Face Cleanser is absolutely one of...,48,14 Jan 2009
4,FRESH Soy Face Cleanser,face-wash-facial-cleanser,FRESH,Soy Face Cleanser,chiquita4011,5,Switched from Purity Made Simple...,I was using Philosophy's Purity Made Simple fo...,42,12 Dec 2008


In [6]:
#drop rows that dont have username or reviews
#drop deplicated reviews

review.dropna(subset=['user', 'long_review'], inplace = True)
review.drop_duplicates(subset = ['long_review'],inplace = True)

In [7]:
review.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138017 entries, 0 to 158207
Data columns (total 10 columns):
full_name       138017 non-null object
Category        138017 non-null object
brand           138017 non-null object
name            138017 non-null object
user            138017 non-null object
stars           138017 non-null int64
short_review    107882 non-null object
long_review     138017 non-null object
helpfulness     138017 non-null int64
time            137585 non-null object
dtypes: int64(2), object(8)
memory usage: 11.6+ MB


In [8]:
#drop product reviews that product has no rating(not enough people rating), 
#or it is a product set, which include more than one product.
droplist = [ 'PHILOSOPHY Purity Cleanser + Microdelivery Wash BFF Set',  'LANEIGE Moisturizing Foam Cleanser',
            'LANCER The Method: Cleanse', 'BOSCIA Resurfacing Treatment Toner with Apple Cider',
            'EVIAN Brumisateur® Natural Mineral Water Facial Spray Travel Trio', "BELIF Aqua Bomb Surf's Up"]
for i in droplist:
    for e in review.full_name:
        if e == i:
            review = review[review.full_name != e ]

In [9]:
review = review.reset_index(drop=True)
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137685 entries, 0 to 137684
Data columns (total 10 columns):
full_name       137685 non-null object
Category        137685 non-null object
brand           137685 non-null object
name            137685 non-null object
user            137685 non-null object
stars           137685 non-null int64
short_review    107655 non-null object
long_review     137685 non-null object
helpfulness     137685 non-null int64
time            137253 non-null object
dtypes: int64(2), object(8)
memory usage: 10.5+ MB


In [10]:
#assign id number to each review
l = []
n = 0
for i in range(len(review)):
    try:
        if review.name[i] == review.name[i+1]:
            l.append(n)
        else:
            l.append(n)
            n = n+1
    except KeyError:
        l.append(n)
review['id'] = l

In [11]:
#repurchase rate
l = []
l1 = []
l2 = []
l3 = []
for i in range(281):
    frequency = review[review.id == i].user.value_counts()
    total = len(frequency)
    repurchase = sum([1 for i in frequency if i >1])
    rate = repurchase/total
    name = list(review[review.id == i].full_name)[0]
    
    l.append(name)
    l1.append(total)
    l2.append(repurchase)
    l3.append(rate)
    
df_repurchase = pd.DataFrame({'product': l, 'total':l1,'repurchase': l2, 'rate': l3})

In [12]:
df_repurchase.sort_values(by = 'rate', ascending = False).head()

,product,total,repurchase,rate
178,ANTHONY Astringent Toner Pads,98,5,0.051020
76,CLINIQUE Rinse-Off Foaming Cleanser,704,27,0.038352
47,PHILOSOPHY The Microdelivery Exfoliating Facia...,2776,99,0.035663
275,SHISEIDO Bio-Performance Advanced Super Revita...,321,11,0.034268
142,MURAD Hydrating Toner,270,9,0.033333


In [13]:
df_repurchase.to_csv('../datasets/repurchase.csv')

In [14]:
review.to_csv('../datasets/review_clean.csv')